In [0]:
!pip install tqdm
  
import time, pandas as pd
import re, tensorflow as tf, numpy as np
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tqdm import tqdm
from tensorflow.python.layers.core import Dense
import os

In [0]:
from google.colab import drive

drive.mount('/content/drive')

In [0]:
with open("/content/drive/My Drive/datasets/datasets/cornell movie-dialogs corpus/movie_lines.txt",encoding="utf-8",errors="ignore") as file:
    l_raw=file.read().split("\n")
    
with open("/content/drive/My Drive/datasets/datasets/cornell movie-dialogs corpus/movie_conversations.txt",encoding="utf-8",errors="ignore")as file:
    conversation_raw=file.read().split("\n")

In [0]:
#dictionary with line id and text
emp_dict={}
for line in l_raw:
    inline=line.split(" +++$+++ ")
    if len(inline) == 5:
        emp_dict[inline[0]]=inline[4]

In [0]:
list_conversation_ids=[]

for line in conversation_raw[:-1]:
    inline=line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    
    list_conversation_ids.append(inline.split(","))

In [0]:
inputs=[] #questions 
targets=[] #answers


for eachid in list_conversation_ids:
    #print(eachid)
    for i in range(len(eachid)-1):
        
        inputs.append(emp_dict[eachid[i]])
        targets.append(emp_dict[eachid[i+1]])
                
            

In [0]:
def data_cleaning(text):
    text=text.lower()
    text=re.sub(r"[-()#\/@{}?!,+.]","",text)#remove unwanted symbols etc from text
    return text

In [0]:
modi_inputs=[] #clean_questions
for text in inputs:
    modi_inputs.append(data_cleaning(text))
    
modi_targets=[] # clean_answers

for text in targets:
    modi_targets.append(data_cleaning(text))
    

inputs=modi_inputs
targets=modi_targets
    
    

In [0]:
size_input=[len(x.split()) for x in inputs]
size_targets=[len(x.split()) for x in targets]
size=size_input + size_targets
min_size=2
max_size=8

In [0]:
#removing answers and questions that are too long as they reduced efficiency in trials
#first for question

inputs_short_real=[]
targets_short_real=[]

i=0
for eachinput in inputs:
    if len(eachinput.split()) >=min_size and len(eachinput.split()) <=max_size:
        if len(targets[i].split()) >=min_size and len(targets[i].split()) <=max_size:
            
            inputs_short_real.append(eachinput)
            targets_short_real.append(targets[i])
        
    i+=1
    

In [0]:
#getting occurances of words and removing rare in the next step
freq_words={}

for eachin in inputs_short_real:
    for word in eachin.split():
        if word not in freq_words:
            freq_words[word]=1
        else:
            freq_words[word]+=1
            

for eachin in targets_short_real:
    for word in eachin.split():
        if word not in freq_words:
            freq_words[word]=1
        else:
            freq_words[word]+=1
                        
            
min_freq=5 #this is an arbitrary value tweak it and compare resutls
count=0
for k,v in freq_words.items():
    if v>=min_freq:
        count+=1    

In [0]:
#we can use nltk tokenizer and lametizer but for computational purpose and time we are not using that


word2vec_inputs={}

num=0

for word,count in freq_words.items():
    if count>=min_freq:
        word2vec_inputs[word]=num
        num+=1
        
        
word2vec_targets={}

num=0

for word,count in freq_words.items():
    if count>=min_freq:
        word2vec_targets[word]=num
        num+=1        

In [0]:
unk_etc_tokens=['<pad>','<eos>','<unk>','<go>']

for eac in unk_etc_tokens:
    word2vec_inputs[eac]=len(word2vec_inputs)+1
    
for eac in unk_etc_tokens:
    word2vec_targets[eac]=len(word2vec_targets)+1    

    
encoder_vocab_size=len(word2vec_inputs)
decoder_vocab_size=len(word2vec_targets)

In [0]:
word2vec_inputs["<pad>"]=0
word2vec_inputs["<unk>"]=1
word2vec_targets["<go>"]=1
word2vec_targets["<pad>"]=0

In [0]:
#now that we have created dictionaries to convert words to vectors we should also create ones that convert
#....vectors to words ( its required when we are testing or deploying the chatbot)
sequence_len=10
dec_sentence_length=12
enc_sentence_length=10
vec2word_inputs={vec:word for word,vec in word2vec_inputs.items()}
vec2word_targets={vec:word for word,vec in word2vec_targets.items()}

In [0]:
import numpy as np
np.shape(inputs_short_real)

(60324,)

In [0]:
inputs_short_real=inputs_short_real[:10000]
targets_short_real=targets_short_real[:10000]

In [0]:
encoder_input=np.zeros([len(inputs_short_real),enc_sentence_length],dtype="int32")
decoder_input=np.zeros([len(targets_short_real),dec_sentence_length],dtype="int32")

In [0]:
for i,(input_data,target_data) in enumerate(zip(inputs_short_real,targets_short_real)):
  line=0
  #print(input_data)
  for char in input_data.split():
    
    char=char.lower()
    try:
      encoder_input[i,line]=word2vec_inputs[char]
    except:
      encoder_input[i,line]=word2vec_inputs["<unk>"]
    line+=1
    
  line=0
  #print(input_data)
  for char in target_data.split():
    
    char=char.lower()
    try:
      decoder_input[i,line]=word2vec_targets[char]
    except:
      decoder_input[i,line]=word2vec_targets["<unk>"]
    line+=1  

In [0]:
class DemoConfig:
    attention_size = 512
    hidden_size = 512
    encoder_embedding_size = 5000
    decoder_embedding_size = 5000
   
    cell = tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell #can choose a basic cpu supported cell as well will only make computation slower, performance will be same
    
    optimizer = tf.train.RMSPropOptimizer
    n_epoch = 1001
    learning_rate = 0.001
    start_token =1# word2vec_targets["<go>"]
    end_token =0# word2vec_targets["<pad>"]

    
    ckpt_dir = './ckpt_dir/'

In [0]:
def wordtovector(incoming,input_token_index=word2vec_inputs,decoder_target=False):
  
  if decoder_target:
    outer=[word2vec_targets["<go>"]] + incoming
    #print(outer)
    #print("enco",len(outer))
    return outer,len(outer)
  
  else:
    #print(np.shape(incoming))
    #print("enco",len(incoming))
    return incoming,len(incoming)



In [0]:
inputs_short_real=np.reshape(encoder_input,[25,400,10]).tolist()
targets_short_real=np.reshape(decoder_input,[25,400,12]).tolist()

In [0]:
class Seq2SeqModel(object):
    def __init__(self, config, mode='training'):
        assert mode in ['training','inference']
        self.mode = mode
        self.hidden_size = config.hidden_size
        self.encoder_embedding_size = config.encoder_embedding_size
        self.decoder_embedding_size = config.decoder_embedding_size
        self.attention_size = config.attention_size
        self.cell = config.cell
        self.optimizer = config.optimizer
        self.n_epoch = config.n_epoch
        self.learning_rate = config.learning_rate
        
        self.start_token = config.start_token
        self.end_token = config.end_token
        self.ckpt_dir = config.ckpt_dir
        
    def add_placeholders(self):
        self.enc_inputs = tf.placeholder(tf.int32,shape=[None, enc_sentence_length],name='input_sentences')

        self.enc_sequence_length = tf.placeholder(tf.int32,shape=[None,],name='input_sequence_length')
        
        if self.mode == 'training':
            self.dec_inputs = tf.placeholder(tf.int32,shape=[None, dec_sentence_length+1],name='target_sentences')#reduced +1

            self.dec_sequence_length = tf.placeholder(tf.int32,shape=[None,],name='target_sequence_length')
            
            self.targets=tf.placeholder(tf.int32,shape=[None,dec_sentence_length])
            
    def add_encoder(self):
        with tf.variable_scope('Encoder') as scope:
            with tf.device('/CPU:0'):
                self.enc_Wemb = tf.get_variable('embedding',initializer=tf.random_uniform([encoder_vocab_size+1, self.encoder_embedding_size]),dtype=tf.float32)

            
            enc_emb_inputs = tf.nn.embedding_lookup(self.enc_Wemb, self.enc_inputs, name='emb_inputs')
            enc_cell = self.cell(self.hidden_size)
            self.enc_outputs, self.enc_last_state = tf.nn.dynamic_rnn(cell=enc_cell,inputs=enc_emb_inputs,sequence_length=self.enc_sequence_length,time_major=False,dtype=tf.float32)
            
    def add_decoder(self):
        with tf.variable_scope('Decoder') as scope:
            with tf.device('/CPU:0'):
                self.dec_Wemb = tf.get_variable('embedding',initializer=tf.random_uniform([decoder_vocab_size+2, self.decoder_embedding_size]),dtype=tf.float32)
            
            
            batch_size = tf.shape(self.enc_inputs)[0]

            dec_cell = self.cell(self.hidden_size)
            
            attn_mech = tf.contrib.seq2seq.BahdanauAttention(num_units=self.attention_size,memory=self.enc_outputs,memory_sequence_length=self.enc_sequence_length,name='LuongAttention')

            
            dec_cell = tf.contrib.seq2seq.AttentionWrapper(cell=dec_cell,attention_mechanism=attn_mech,attention_layer_size=self.attention_size,name='Attention_Wrapper')
            
            initial_state=dec_cell.zero_state(dtype=tf.float32, batch_size=batch_size)
            output_layer = Dense(decoder_vocab_size+2, name='output_projection')
            
            if self.mode == 'training':

              
                max_dec_len = tf.reduce_max(self.dec_sequence_length-1, name='max_dec_len')#+1 excluded from dec_sequence_length

                dec_emb_inputs = tf.nn.embedding_lookup(self.dec_Wemb, self.dec_inputs, name='emb_inputs')
        
                training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_emb_inputs,sequence_length=self.dec_sequence_length+1,time_major=False,name='training_helper')

                training_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,helper=training_helper,initial_state=initial_state,output_layer=output_layer) 

                train_dec_outputs, train_dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,output_time_major=False,impute_finished=True,maximum_iterations=max_dec_len)
                

                logits = tf.identity(train_dec_outputs.rnn_output, name='logits')
                
                
                targets=self.targets
                
                #targets = tf.slice(self.dec_inputs, [0, 0], [-1, max_dec_len], 'targets')
                #print(targets)
                
                
                masks = tf.sequence_mask(self.dec_sequence_length+1, max_dec_len, dtype=tf.float32, name='masks')
                
                
                self.batch_loss = tf.contrib.seq2seq.sequence_loss(logits=logits,targets=targets,weights=masks,name='batch_loss')
                
                
                self.valid_predictions = tf.identity(train_dec_outputs.sample_id, name='valid_preds')
            
            elif self.mode == 'inference':
            
                start_tokens = tf.tile(tf.constant([self.start_token], dtype=tf.int32), [batch_size], name='start_tokens')
            
                inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=self.dec_Wemb,start_tokens=start_tokens,end_token=self.end_token)
                
                inference_decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,helper=inference_helper,initial_state=initial_state,output_layer=output_layer)
                
                infer_dec_outputs, infer_dec_last_state, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,output_time_major=False,impute_finished=True,maximum_iterations=dec_sentence_length)
                
                self.predictions = tf.identity(infer_dec_outputs.sample_id, name='predictions')
               
        
    def add_training_op(self):
        self.training_op = self.optimizer(self.learning_rate, name='training_op').minimize(self.batch_loss)
        
    def save(self, sess, var_list=None, save_path=None):
        print('Saving model at {save_path}')
        if hasattr(self, 'training_variables'):
            var_list = self.training_variables
        saver = tf.train.Saver(var_list)
        saver.save(sess, save_path, write_meta_graph=False)
        
    def restore(self, sess, var_list=None, ckpt_path=None):
        if hasattr(self, 'training_variables'):
            var_list = self.training_variables
        self.restorer = tf.train.Saver(var_list)
        self.restorer.restore(sess, ckpt_path)
        print('Restore Finished!')
        
    def summary(self):
        summary_writer = tf.summary.FileWriter(
            logdir=self.ckpt_dir,
            graph=tf.get_default_graph())
        
    def build(self):
        self.add_placeholders()
        self.add_encoder()
        self.add_decoder()

    def train(self, sess, data, from_scratch=False,
              load_ckpt=None, save_path=None):
        
        
        if from_scratch is False and os.path.isfile(load_ckpt):
            self.restore(sess, ckpt_path=load_ckpt)
    
        
        self.add_training_op()
        
        sess.run(tf.global_variables_initializer())
        
        inputs_data_real, targets_data_real = data
        loss_history = []
        
        for epoch in tqdm(range(self.n_epoch)):

            all_preds = []
            epoch_loss = 0
            for input_batch, target_batch in zip(inputs_data_real, targets_data_real):
                input_batch_tokens = []
                target_batch_tokens = []
                enc_sentence_lengths = []
                dec_sentence_lengths = []
                actual_target_tokens=[]
                
                for input_sent in input_batch:
                  tokens, sent_len = wordtovector(input_sent)
                  input_batch_tokens.append(tokens)
                  enc_sentence_lengths.append(sent_len)

                for target_sent in target_batch:
                  tokens, sent_len = wordtovector(target_sent,input_token_index=word2vec_targets,decoder_target=True)
                                 
                  target_batch_tokens.append(tokens)
                  dec_sentence_lengths.append(sent_len)
                  
                for target_sent in target_batch:
                  tokens, sent_len = wordtovector(target_sent)
                  #print(sent_len)
                                 
                  actual_target_tokens.append(tokens)
                   
                  
                #print()  
                #print("inpu",input_batch_tokens)  
                #print("t",target_batch_tokens)
                # Evaluate 3 ops in the graph
                # => valid_predictions, loss, training_op(optimzier)
                batch_preds, batch_loss, _ = sess.run(
                    [self.valid_predictions, self.batch_loss, self.training_op],
                    feed_dict={
                        self.enc_inputs: input_batch_tokens,
                        self.enc_sequence_length: enc_sentence_lengths,
                        self.dec_inputs: target_batch_tokens,
                        self.dec_sequence_length: dec_sentence_lengths,
                        self.targets:actual_target_tokens,
                    })
                # loss_history.append(batch_loss)
                epoch_loss += batch_loss
                all_preds.append(batch_preds)
                
            loss_history.append(epoch_loss)

            
            if epoch % 2 == 0:
                print('Epoch', epoch)
                #for input_batch, target_batch, batch_preds in zip(inputs_data_real, targets_data_real, all_preds):
                 #   for input_sent, target_sent, pred in zip(input_batch, target_batch, batch_preds):
                      
                        #pass
                 #       print('\tInput: {input_sent}')
                      #print('\tPrediction:'pred)
                 #       print('\tTarget:, {%s}'%target_sent)
                print('\tepoch loss: {epoch_loss:%s}\n'%epoch_loss)
                
        if save_path:
            self.save(sess, save_path=save_path)

        return loss_history
    
    def inference(self, sess, data, load_ckpt):
        
        self.restore(sess, ckpt_path=load_ckpt)
        
        input_batch, target_batch = data

        batch_preds = []
        batch_tokens = []
        batch_sent_lens = []
        
        for input_sent in [input_batch]:
            tokens, sent_len = wordtovector(input_sent)
            batch_tokens.append(tokens)
            batch_sent_lens.append(sent_len)
            
        batch_preds = sess.run(
            self.predictions,
            feed_dict={
                self.enc_inputs: batch_tokens,
                self.enc_sequence_length: batch_sent_lens,
            })

        for input_sent, target_sent, pred in zip(input_batch, target_batch, batch_preds):
          language="en"
          pred= " ".join([vec2word_targets[x] for x in pred])
          print(pred)
          #myobj=gTTS(text=pred,lang=language,slow=False)

          #myobj.save("new.mp3")
          #temp=easygui.msgbox(title="Tiger the chatbot",msg=pred)
          #ipd.Audio("new.mp3",autoplay=True)

In [0]:
!cp -r 'drive/My Drive/ckpt_dir/' ckpt_dir/
!ls ckpt_dir/

checkpoint  epoch_1001_attention.data-00000-of-00001
ckpt_dir    epoch_1001_attention.index


In [0]:
#tf.reset_default_graph()     

"""
with tf.Session() as sess:
    config = DemoConfig()
    model = Seq2SeqModel(config, mode='training')
    model.build()
    data = (inputs_short_real, targets_short_real)
    loss_history = model.train(sess, data, load_ckpt=model.ckpt_dir+f'epoch_{model.n_epoch}_attention', save_path=model.ckpt_dir+f'epoch_{model.n_epoch}_attention')
    
"""
    

  0%|          | 1/1001 [00:17<4:57:37, 17.86s/it]

Epoch 0
	epoch loss: {epoch_loss:197.46082162857056}



  0%|          | 3/1001 [00:49<4:41:36, 16.93s/it]

Epoch 2
	epoch loss: {epoch_loss:87.03668117523193}



  0%|          | 5/1001 [01:22<4:33:52, 16.50s/it]

...

Epoch 996
	epoch loss: {epoch_loss:1.814487874507904}



100%|█████████▉| 999/1001 [4:22:48<00:31, 15.66s/it]

Epoch 998
	epoch loss: {epoch_loss:1.7581388428807259}



100%|██████████| 1001/1001 [4:23:19<00:00, 15.67s/it]

Epoch 1000
	epoch loss: {epoch_loss:1.7533504962921143}

Saving model at {save_path}


In [0]:
raw=np.zeros([2,10])
text_in=input("say something in lower case \n")
o=0
for chars in text_in.split():
  raw[0,o]=word2vec_inputs[chars]
  raw[1,o]=word2vec_inputs[chars]
  o+=1
config = DemoConfig()  
tf.reset_default_graph()
with tf.Session() as sess:
    model = Seq2SeqModel(config, mode='inference')
    model.build()
    for input_batch, target_batch in zip(raw, [[111 ,2333]]):
        data = (input_batch, target_batch)
        model.inference(sess, data, load_ckpt=model.ckpt_dir+f'epoch_{model.n_epoch}_attention')

say something in lower case 
what do you like
INFO:tensorflow:Restoring parameters from ./ckpt_dir/epoch_1001_attention
Restore Finished!
it's me <pad>


In [0]:
!mv 'ckpt_dir' 'drive/My Drive/' 

In [0]:
!cp -r 'drive/My Drive/ckpt_dir/' ckpt_dir/
ls ckpt_dir/

In [0]:
ls ckpt_dir/

checkpoint                                epoch_1001_attention.index
epoch_1001_attention.data-00000-of-00001


In [0]:
!mv 'drive/My Drive/ckpt_dir/' ckpt_dir